In [1]:
%matplotlib notebook
%load_ext autoreload
%pwd

'/ocean/projects/asc170022p/mtragoza/lung-project/notebooks'

In [2]:
import sys
import pandas as pd
sys.path.append('../../param_search')
import param_search as ps

## Setup experiment

[[Setup](#Setup-experiment)] [[Submit](#Submit-jobs)] [[Monitor](#Monitor-jobs)] [[Analyze](#Analyze-results)]

In [3]:
# define a job template and name format
template = '''\
#!/bin/bash
#SBATCH --job-name={job_name}
#SBATCH --account=asc170022p
#SBATCH --partition=GPU-shared
#SBATCH --gres=gpu::1
#SBATCH --mem=64000M
#SBATCH -x v034
#SBATCH --time=48:00:00
#SBATCH -o %J.stdout
#SBATCH -e %J.stderr
#SBATCH --mail-type=all

hostname
pwd
module load anaconda3
conda activate /ocean/projects/asc170022p/mtragoza/mambaforge/envs/lung-project
nvidia-smi

python ../../../train.py \\
    --data_name emory \\
    --data_root /ocean/projects/asc170022p/shared/Data/4DLungCT/Emory/ \\
    --mask_roi {mask_roi} \\
    --mesh_version {mesh_version} \\
    --test_case {test_case} \\
    --test_phase {test_phase} \\
    --rho_value {rho_value} \\
    --conv_channels {conv_channels} \\
    --output_func {output_func} \\
    --interp_size {interp_size} \\
    --save_prefix {job_name} 

echo Done
'''
name_format = 'train__emory__{test_case}__{test_phase}__{rho_value}__{output_func}'

In [4]:
param_space = ps.ParamSpace(
    mask_roi='lung_regions2',
    mesh_version=11,
    test_case=[None], #['Case1Pack', 'Case2Pack', 'Case3Pack', 'Case4Pack', 'Case5Pack', 'Case6Pack', 'Case7Pack', 'Case8Deploy', 'Case9Pack'],
    test_phase=[0], # 10, 20, 30, 40, 50, 60, 70, 80, 90],
    rho_value=[0, 1000, 'anat'],
    conv_channels=[32],
    interp_size=[5],
    output_func=['relu', 'exp', 'softplus'],
)

for p in param_space:
    print(name_format.format(**p))

print(len(param_space))

train__emory__None__0__0__relu
train__emory__None__0__0__exp
train__emory__None__0__0__softplus
train__emory__None__0__1000__relu
train__emory__None__0__1000__exp
train__emory__None__0__1000__softplus
train__emory__None__0__anat__relu
train__emory__None__0__anat__exp
train__emory__None__0__anat__softplus
9


## Submit jobs

[[Setup](#Setup-experiment)] [[Submit](#Submit-jobs)] [[Monitor](#Monitor-jobs)] [[Analyze](#Analyze-results)]

In [ ]:
expt_name = [
    '2024-11-22__phantom__250',
    '2024-11-22__phantom__250__resub',
    '2024-11-30__emory__phase',
    '2024-12-02__emory__interp_size',
    '2024-12-03__emory__gpu_shared',
    '2024-12-07__emory__interface',
    '2024-12-08__emory__lung_regions2',
    '2024-12-08__emory__clamp'
][-1]
expt_name

In [ ]:
do_submit = True
if do_submit:
    jobs = ps.submit(template, name_format, param_space, work_dir=expt_name)
    jobs.to_csv(f'{expt_name}.jobs')

## Monitor jobs

[[Setup](#Setup-experiment)] [[Submit](#Submit-jobs)] [[Monitor](#Monitor-jobs)] [[Analyze](#Analyze-results)]

In [ ]:
jobs = pd.read_csv(f'{expt_name}.jobs', index_col=0)
status = ps.status(jobs, parse_stderr=True)
status

In [ ]:
status['job_state'] = status['job_state'].fillna('DONE')
status['stderr'] = status['stderr'].fillna('N/A')
status.groupby(['job_state', 'stderr'])[['job_name']].count()

In [ ]:
status.iloc[0]

## Analyze results

[[Setup](#Setup-experiment)] [[Submit](#Submit-jobs)] [[Monitor](#Monitor-jobs)] [[Analyze](#Analyze-results)]

In [ ]:
m = ps.metrics(jobs, sep=',')
m

In [ ]:
m.groupby(['job_name'])[['epoch']].max()

In [ ]:
unfinished_jobs = d[d.epoch < 100]
unfinished_jobs

In [ ]:
m.groupby(['job_name'])[['batch']].max()

In [ ]:
m.columns

In [ ]:
%autoreload
fig = ps.plot(
    m[(m.phase == 'train') & (m.epoch > 190) & (m.conv_channels > 8)],
    x=['dummy_targets', 'conv_channels', 'output_func'],
    y=['u_error', 'e_error', 'e_anat_corr', 'CTE'],
    hue=None,
    legend=True,
    legend_kws=dict(bbox_to_anchor=(0, -0.2)),
    tight=True,
    height=2.25, width=2.75
)

In [ ]:
%autoreload
fig = ps.plot(
    m[(m.phase == 'train') & (m.epoch > 190) & (m.conv_channels > 8)],
    x=['dummy_targets', 'conv_channels', 'output_func'],
    y=['u_error', 'e_error', 'e_anat_corr', 'CTE'],
    hue=None,
    legend=True,
    legend_kws=dict(bbox_to_anchor=(0, -0.2)),
    tight=True,
    height=2.25, width=2.75
)

In [ ]:
%autoreload
fig = ps.plot(
    m[(m.phase == 'test') & (m.epoch > 190) & (m.conv_channels > 8)],
    x=['dummy_targets', 'conv_channels', 'output_func'],
    y=['u_error', 'e_error', 'e_anat_corr', 'CTE'],
    hue=None,
    legend=True,
    legend_kws=dict(bbox_to_anchor=(0, -0.2)),
    tight=True,
    height=2.25, width=2.75
)

In [ ]:
m[(m.phase == 'test') & (m.epoch > 190) & (m.conv_channels > 8)].groupby(['dummy_targets', 'conv_channels', 'output_func', 'phase'])[['u_error', 'e_error', 'e_anat_corr', 'CTE']].describe()